In [1]:
import numpy as np
import keras
import pandas as pd
import joblib
from keras import Sequential
from first_test import EVENT_DICT
import first_test

2025-04-27 18:07:22.679293: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 18:07:22.679653: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 18:07:22.681926: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 18:07:22.688811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745798842.700305   20926 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745798842.70

In [2]:
dev_x = np.load('dwt_najafi/dev_naj_x.npy', allow_pickle=True)
dev_y = np.load('dwt_najafi/dev_naj_y.npy', allow_pickle=True)
eval_x = np.load('dwt_najafi/eval_naj_x.npy', allow_pickle=True)
eval_y = np.load('dwt_najafi/eval_naj_y.npy', allow_pickle=True)
train_x = np.load('dwt_najafi/train_naj_x.npy', allow_pickle=True)
train_y = np.load('dwt_najafi/train_naj_y.npy', allow_pickle=True)

In [9]:
print(dev_y)
print(train_x.shape)

[12 12 12 ...  2  2  2]
(4663,)
<class 'list'>


In [10]:
print(type(train_x[0][0][0]))
print()

<class 'list'>


In [ ]:
"""dev_y.apply(lambda x: EVENT_DICT[x])
eval_y.apply(lambda x: EVENT_DICT[x])
train_y.apply(lambda x: EVENT_DICT[x])"""

In [3]:
def create_naj_model():
    model = Sequential([
        keras.Input(shape=(880,)),
        keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True)),
        keras.layers.Dense(200, activation='linear'),
        keras.layers.Dense(200, activation='linear'),
        keras.layers.Dense(12, activation='softmax'),
    ])
    return model
model = create_naj_model()
model.summary()

E0000 00:00:1745797927.337973   19826 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1745797927.341935   19826 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


ValueError: Input 0 of layer "bidirectional" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 880)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y, epochs=30, batch_size=150, validation_data=(eval_x, eval_y))